#Import Python Packages

In [1]:
import sys
sys.path.insert(0,'/global/homes/b/bpb/metatlas/metatlas/helpers/' )
import dill2plots as dp
%matplotlib inline

('Metatlas live in ', '/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages/metatlas')
you're running on d9d4d6ec916d at 172.17.0.9 


:0: FutureWarning: IPython widgets are experimental and may change in the future.


#Make blank file-info sheet

Get files from database

In [10]:
files = dp.get_metatlas_files(experiment = '%hedlund%',name = '%hilic%',most_recent = True)

Save the files to an empty fileinfo sheet

In [9]:
dp.make_empty_fileinfo_sheet('/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/hedlund_empty_fileinfo.tab',files)

#Create metatlas groups from filled in file-info sheet

In [18]:
dp = reload(dp)
g = dp.make_groups_from_fileinfo_sheet('/global/homes/b/bpb/notebooks/atlas_finfo_to_be_loaded/FileInfoSheet_BHedlund_AllFiles_20160413 (1).txt',store=False)

#Create a new Atlas in metatlas from a spreadsheet

In [5]:
dp = reload(dp)
names = dp.make_atlas_from_spreadsheet('/global/homes/b/bpb/Downloads/201500826_library_qxqct_hilic_neg_V2.txt',
                                       'temp',
                                       filetype='tab',
                                       sheetname='',
                                       polarity = 'positive',
                                       store=False)

0 209.1183184 1.3 1.7
1 209.1183184 1.8 2.0
2 89.02441806 7.5 8.0
3 218.1033967 7.4 7.7
4 133.0142473 16.0 20.6
5 179.0561121 8.1 8.9
6 179.0561121 9.7 10.7
7 149.0455474 5.1 6.3
8 149.0455474 10.7 11.1
9 149.0455474 7.0 8.6
10 163.0611975 5.1 6.1
11 341.1089355 12.1 12.4
12 341.1089355 12.5 13.0
13 130.0509672 10.7 11.2
14 173.0091619 17.1 17.6
15 173.0091619 17.9 18.6
16 179.0561121 12.8 13.2
17 220.0826612 7.3 7.7
18 353.0878062 9.7 12.0
19 179.0349827 2.0 2.3
20 147.0451535 2.2 2.4
21 145.0142473 15.0 15.3
22 242.0782445 7.4 7.8
23 115.0036826 15.7 16.0
24 174.0560525 4.2 5.5
25 154.0622005 14.3 15.0
26 130.0873527 7.5 8.0
27 130.0873527 7.2 7.5
28 104.0353171 12.5 13.1
29 180.0666172 9.8 10.2
30 116.0717026 8.7 9.1
31 263.1288831 1.3 1.8
32 263.1288831 1.9 2.2
33 102.0560525 9.9 10.3
34 175.0400681 1.4 1.6
35 116.0717026 12.0 12.3
36 134.0472192 4.1 4.7
37 243.0808871 7.1 7.5
38 181.0717622 9.5 9.8
39 193.0506328 3.4 4.7
40 209.0302913 16.8 17.6
41 74.02475241 12.0 12.4
42 75.0087

#Select groups of files to operate on

In [2]:
groups = dp.select_groups_for_analysis(name = '%hedlund%neg%',most_recent = True, remove_empty = True, filter_list = [])
for g in groups:
    print g.name

0 20160413_Bhedlund_pHILIC_NEG_GBS_Med_inc_Set1 2016-04-14 17:59:30
1 20160413_Bhedlund_pHILIC_NEG_Blank_Set2 2016-04-13 22:34:24
2 20160413_Bhedlund_pHILIC_NEG_PR_End_Set1 2016-04-14 17:59:36
3 20160413_Bhedlund_pHILIC_NEG_JKG1_PR_Set1 2016-04-14 17:59:33
4 20160413_Bhedlund_pHILIC_NEG_GBS_32OE_Ctrl_Set1 2016-04-14 17:59:32
5 20160413_Bhedlund_pHILIC_NEG_JAD2_GBS_4OE_Set1 2016-04-14 17:59:37
6 20160413_Bhedlund_pHILIC_NEG_GBS_32OE_inc_Set2 2016-04-13 22:34:26
7 20160413_Bhedlund_pHILIC_NEG_G233_BMR_Set1 2016-04-14 17:59:19
8 20160413_Bhedlund_pHILIC_NEG_BMR_End_Set1 2016-04-14 17:59:39
9 20160413_Bhedlund_pHILIC_NEG_JAD2_GBS_32OE_Set2 2016-04-13 22:34:27
10 20160413_Bhedlund_pHILIC_NEG_JAD2_GBS_32OE_Set1 2016-04-14 17:59:20
11 20160413_Bhedlund_pHILIC_NEG_GBS_32OE_Ctrl_Set2 2016-04-13 22:34:26
12 20160413_Bhedlund_pHILIC_NEG_GBS_Med_Ctrl_Set1 2016-04-14 17:59:27
13 20160413_Bhedlund_pHILIC_NEG_JAD2_GBS_Set1 2016-04-14 17:59:39
14 20160413_Bhedlund_pHILIC_NEG_PR_Beg_Set1 2016-04-14 17:

#Select Atlas to use

In [10]:
dp = reload(dp)
atlas = dp.get_metatlas_atlas(name='%library%',do_print = True)


0 201500826_library_qxqct_hilic_neg_V2 2016-04-25 23:08:19


In [12]:
myAtlas = atlas[0]
myAtlas.name

u'201500826_library_qxqct_hilic_neg_V2'

#Export Atlas To Spreadsheet

In [18]:
dp = reload(dp)
df = dp.export_atlas_to_spreadsheet(myAtlas,'/global/homes/b/bpb/Downloads/kz_neg_mode_atlas.csv')
df.head()

,name,inchi,mono_isotopic_molecular_weight,creation_time,formula,last_modified,unique_id,username,label,rt_min,...,common_name,synonyms,pubchem_compound_id,lipidmaps_id,metacyc_id,hmdb_id,img_abc_id,chebi_id,kegg_id,metatlas_inchi
0,(-)-jasmonate,InChI=1S/C12H18O3/c1-2-3-4-5-10-9(8-12(14)15)6...,210.125595,1444322253,C12H18O3,1461625699,a1c63d0ef3414e8ea59fefcb9933d376,katezh,jasmonic acid (peak1),1.3,...,(-)-7-iso-jasmonate///(+)-7-iso-jasmonate///(-...,(+)-7-iso-jasmonic acid///(+)-7-iso-jasmonate/...,7251182///7251176///7251179///7251181,nan,CPD-734///CPD-5941///CPD-731///CPD-5943,nan,nan,CHEBI:58431,C08491///C16317,InChI=1S/C12H18O3/c1-2-3-4-5-10-9(8-12(14)15)6...
1,(-)-jasmonate,InChI=1S/C12H18O3/c1-2-3-4-5-10-9(8-12(14)15)6...,210.125595,1444322253,C12H18O3,1444322294,b791cccd7bd648c58414afd6888981f4,bpb,jasmonic acid (peak2),1.8,...,(-)-7-iso-jasmonate///(+)-7-iso-jasmonate///(-...,(+)-7-iso-jasmonic acid///(+)-7-iso-jasmonate/...,7251182///7251176///7251179///7251181,nan,CPD-734///CPD-5941///CPD-731///CPD-5943,nan,nan,CHEBI:58431,C08491///C16317,InChI=1S/C12H18O3/c1-2-3-4-5-10-9(8-12(14)15)6...
2,(<i>R</i>)-lactate,"InChI=1S/C3H6O3/c1-2(4)3(5)6/h2,4H,1H3,(H,5,6)...",90.031693,1444322244,C3H6O3,1461625699,d0bd47f557a048b98b3c3803826ba957,katezh,lactic acid,7.5,...,(S)-lactate///(S)-lactate///(R)-lactate///(R)-...,Monosodium lactate///(S)-lactate///Lactic acid...,5460179///5460161,nan,L-LACTATE///D-LACTATE,HMDB15671,nan,CHEBI:75228///CHEBI:24996///CHEBI:16651///CHEB...,C00186///C01432///C00256,"InChI=1S/C3H6O3/c1-2(4)3(5)6/h2,4H,1H3,(H,5,6)"
3,(<i>R</i>)-pantothenate,"InChI=1S/C9H17NO5/c1-9(2,5-11)7(14)8(15)10-4-3...",219.110672,1444322177,C9H17NO5,1461625699,b8358b4671f94b6d8a6a6ccce7385046,katezh,pantothenic acid,7.4,...,(R)-pantothenate///(R)-pantothenate///pantothe...,"D-pantothenic acid///D(+)-N-(2,4-Dihydroxy-3,3...",167945,nan,PANTOTHENATE,HMDB00210,nan,CHEBI:29032///CHEBI:16454///CHEBI:31345///CHEB...,C00864///C00864///C12276,"InChI=1S/C9H17NO5/c1-9(2,5-11)7(14)8(15)10-4-3..."
4,(R)-malate,"InChI=1S/C4H6O5/c5-2(4(8)9)1-3(6)7/h2,5H,1H2,(...",134.021530,1444322243,C4H6O5,1461625699,d5cbfb4a45dd44c6a68328698a747d14,katezh,malic acid,16.0,...,disodium (S)-malate///Malic acid///(S)-malate(...,(S)-malate///(S)-disodium malate///Deoxytetrar...,5459794///5459792,nan,CPD-660///MAL,HMDB00744,nan,CHEBI:6650///CHEBI:15589///CHEBI:15588///CHEBI...,C00149///C00497///C00711,"InChI=1S/C4H6O5/c5-2(4(8)9)1-3(6)7/h2,5H,1H2,(..."


#Get Data and Save it

In [ ]:
# get and pickle everything This is MSMS, raw MS1 datapoints, compound, group info, and file info
# combine positive and negative mode atlas, by join atlases together vs two compound references
# typically file-groups are NOT split by polarity, files with un-matched polarity will be discarded for analysis of an identification
dp = reload(dp)
output_filename = '/global/homes/b/bpb/data/hedlund/20160425_Bhedlund_pHILIC_NEG_AllGroups_master_atlas.pkl'
data = dp.get_data_for_groups_and_atlas(groups,myAtlas,output_filename,use_set1 = True)

0 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH36.h5
0 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH37.h5
0 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH38.h5
0 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH39.h5
0 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSilva_BHedlund_chloroflexi_NEG/20150322_pHILIC_NEG_MSMS_BH40.h5
1 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSILVA_BHedlund_chloroflexi_NEG/20151124_pHILIC_NEG_MSMS_BH_GBSInc_1_blank.h5
1 21 /global/project/projectdirs/metatlas/raw_data/lpsilva/20150406_LPSILVA_BHedlund_chloroflexi_NEG/20151124_pHILIC_NEG_MSMS_BH_IncCtrl_4_blank.h5
2 21 /global/project/projectdirs/me

#Load a pre-existing datafile

In [7]:
my_file = output_filename #'/global/homes/b/bpb/notebooks/MetAtlas_Workflow_20151123/20160420_Bhedlund_pHILIC_NEG_AllGroups_master_atlas.pkl'
project_label = '/global/homes/b/bpb/data/hedlund/20160420_Bhedlund_pHILIC_NEG_AllGroups_master_atlas'
# data = dp.get_data(my_file)

#Make Supplementary Tables

In [8]:
peak_height = dp.make_output_dataframe(input_fname = my_file, fieldname='peak_height' , output_loc=project_label+'/sheets')

#Error bar

In [9]:
dp.plot_errorbar_plots(peak_height,input_fname = my_file, output_loc=project_label+'/error_bar_peak_height')

#Make Chromatograms

In [10]:
dp = reload(dp)
dp.plot_all_compounds_for_each_file(input_fname = my_file, nCols = 12, scale_y=True , output_loc=project_label+'/all_compounds_each_file')
dp.plot_all_files_for_each_compound(input_fname = my_file, nCols = 12, scale_y=True , output_loc=project_label+'/all_files_each_compound')

7
74
nrows =  8
length of ymax is  5620


#Make Identification Figures

In [11]:
dp.make_identification_figure(input_fname = my_file, output_loc=project_label+'/identification')